In [2]:
from sklearn.model_selection import KFold
from sklearn import metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression
from sklearn import ensemble
df = pd.read_csv('data.csv')
X = df[['Weight','PCV','PCV\ndonor','Volume']]
# print(X.columns)
Vet = df[['PCV_target']]
# X = df[['Weight','PCV','PCV\ndonor','Volume','WBC','HGB','RBC','MCV','MCHC','MCH','SEGS','LYMPH']]
# X = df[['Weight', 'PCV', 'PCV\ndonor', 'Volume', 'WBC', 'PLT\n______', 'PLATELETS', 'HGB', 'RBC', 'MCV', 'MCHC', 'MCH',
#         'SEGS', 'LYMPH', 'MONO', 'PROTEIN (REFRACT)', 'RDW']]
y = df['PCV_afterdonation']
column_name = X.columns
X = np.array(X)
# y = df['attack']
y = np.array(y)
Vet = np.array(Vet)


scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)


kf = KFold(n_splits=10)
kf.get_n_splits(X)

RMSE=[]

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#     lm = LinearRegression()
#     lm.fit(X_train,y_train)
    model = ensemble.GradientBoostingRegressor(random_state=1)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    temp_RMSE = np.sqrt(metrics.mean_squared_error(y_test, predictions))
    RMSE.append(temp_RMSE)
#     print(temp_RMSE)


    
print ('Average RMSE 10 fold cross validation Xg: %f'%np.mean(RMSE))


RMSE=[]
# print("leave one out")

from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
loo.get_n_splits(X)

# print(loo)
coef_list = []
for train_index, test_index in loo.split(X):
#    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#    print(X_train, X_test, y_train, y_test)

    model = ensemble.GradientBoostingRegressor(random_state=1)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    temp_RMSE = np.sqrt(metrics.mean_squared_error(y_test, predictions))
    RMSE.append(temp_RMSE)
#     print(lm.coef_)
#     coef_list.append(lm.coef_)
#     print(temp_RMSE)
print ('Average Xg Leave one out : %f'%np.mean(RMSE))
print ('RMSE Vet : %f'%np.mean(np.sqrt(metrics.mean_squared_error(Vet, y))))
print (column_name)
# print ('Average Coef')
# coef_list = np.matrix(coef_list)
# coef_list = np.absolute(coef_list)
# # print(column_name)
# x = coef_list.mean(0)
# # print (x)
# array_x = np.array(x)
# # print(array_x)

# coeff_df = pd.DataFrame(array_x[0],column_name,columns=['Coef'])
# coeff_df = coeff_df.sort_values(by=['Coef'], ascending=False)
# print(coeff_df)

Average RMSE 10 fold cross validation Xg: 6.018278
Average Xg Leave one out : 5.134629
RMSE Vet : 7.503460
Index(['Weight', 'PCV', 'PCV\ndonor', 'Volume'], dtype='object')
